In [1]:
import tweepy as tw
import pandas as pd
import pickle

In [2]:
# Save Bearer Token
BEARER_TOKEN="AAAAAAAAAAAAAAAAAAAAAE0nfwEAAAAAY2LlNZkDVkXKxvcg%2B%2FKvbVyKEgw%3DFIAmWW57LBCnqyCenMTM68H6Kg3hsuKrvXsr5O34Y8bCtQKo7K"
client = tw.Client(BEARER_TOKEN, wait_on_rate_limit=True)

In [3]:

#Define the search
search_words = "AAPL -is:retweet -is:reply lang:en"

date_start = "2022-10-02T10:00:00-05:00"
date_end = "2022-10-05T10:00:00-05:00"
user_fields=['public_metrics']
tweet_fields=['created_at','entities','geo','id','lang', 'public_metrics',
              'source','text']

In [ ]:
#Get Daily Tweet Counts
def get_tweet_counts(query,start,end):
   counts = client.get_all_tweets_count(query=query,end_time=end,
                                        start_time=start, granularity='day')
   daily_counts = pd.DataFrame(columns = ["Day", "Tweet Volume"])
   for count in counts.data:
      c = pd.DataFrame({"Day" : count["start"],
                        "Tweet Volume" : count["tweet_count"]},index=[0])
      daily_counts = pd.concat([daily_counts, c] , ignore_index=True)
   return daily_counts

In [ ]:
get_tweet_counts(search_words,date_start,date_end)

In [ ]:
def get_tweets(query, start, end):
   user_fields=['public_metrics'],
   tweet_fields=['context_annotations','created_at','entities','geo',
                 'id','lang', 'public_metrics','source','text']

   return client.search_all_tweets(query,
                                   end_time=end,
                                   start_time=start,
                                   max_results=20,
                                   sort_order="relevancy",
                                   expansions=['author_id'],
                                   user_fields=user_fields,
                                   tweet_fields=tweet_fields)

In [ ]:
def tweets_to_df(tweets):
   # Create Tweets DF
   tweets_df = pd.DataFrame(columns = ["ID", "Text", "Created at", "Author ID", 
                                       "Retweet Count","Reply Count","Like Count",
                                       "Quote Count"])
   #Iterate over tweets collected and store the fields in helper dataframe
   for tweet in tweets:
      df2 = pd.DataFrame({'ID': tweet.id,
                           "Text":tweet.text,
                           "Author ID":tweet.author_id,
                           "Created at":tweet.created_at,
                           "Retweet Count":tweet.public_metrics["retweet_count"], 
                           "Reply Count":tweet.public_metrics["reply_count"],
                           "Like Count":tweet.public_metrics["like_count"], 
                           "Quote Count":tweet.public_metrics["quote_count"]
                           },index=[0])
      # Concat the two dataframes
      tweets_df = pd.concat([tweets_df, df2] , ignore_index=True)

            # Create Users DF    
   users_df = pd.DataFrame(columns = ["User_ID", "Name", "Username"])

   # Do the same for user fields
   for user in tweets.includes["users"]:
      df2 = pd.DataFrame({'User_ID': user.id,
                           "Name":user.name,
                           "Username":user.username
                           },index=[0])
      users_df = pd.concat([users_df, df2] , ignore_index=True)

   return tweets_df.merge(users_df, left_on="Author ID", right_on='User_ID')

In [ ]:
def get_user_fields(df):
   # Fetch user fields
   ids = list(df["Author ID"].unique())
   users = client.get_users(ids=ids, user_fields=[
                           'name', 'username', 'description', 'public_metrics'])
   users_metrics = pd.DataFrame(columns = ["User_ID", "Name", "Username", 
                                          "User Description", "Followers Count",
                                          "Following Count", "Tweet Count",
                                          "Listed Count"])
   for user in users.data:
      df2 = pd.DataFrame({'User_ID': user.id,
                           "Name":user.name,
                           "Username":user.username,
                           "User Description":user.description,
                           "Followers Count": user.public_metrics['followers_count'], 
                           "Following Count": user.public_metrics['following_count'],
                           "Tweet Count": user.public_metrics['tweet_count'], 
                           "Listed Count": user.public_metrics['listed_count']                       
                           },index=[0])
      users_metrics = pd.concat([users_metrics, df2] , ignore_index=True)
   df = df.merge(users_metrics, left_on="Author ID", right_on='User_ID',
            suffixes=('', '_r'))
   return df.drop(df.filter(regex='_r').columns, axis=1)


In [ ]:
tweets = get_tweets(search_words,date_start,date_end)

In [ ]:
dfz = tweets_to_df(tweets)
dfz.head()

In [ ]:
dfx = dfz.copy()
dfy = get_user_fields(dfx)
dfy

In [ ]:
tweets_df=pd.read_csv("../Data/tweet_sample.csv")

In [4]:
from searchtweets import ResultStream, gen_request_parameters, load_credentials, read_config
from datetime import datetime, timedelta
import time
import pickle
import argparse
from datetime import timedelta

# define date range function
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)


def save_object(obj, filename):
    with open(filename, 'ab') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

In [5]:
#Define the search
search_words = "AAPL -is:retweet -is:reply lang:en"

date_start = "2022-10-02"
date_end = "2022-10-03"
user_fields= ",".join(['public_metrics'])
tweet_fields= ",".join(['created_at','geo','id','lang', 'public_metrics',
              'source','text'])

# get waittime
waittime = 45

In [6]:

# load twitter keys
search_creds = load_credentials('.twitter_keys.yaml',
                               yaml_key = 'search_tweets_v2',
                               env_overwrite = False)
# load configuration for search query
config = read_config('search_config.yaml')

# set interval to loop through
start_date = datetime.strptime(date_start, '%Y-%m-%d').date()
end_date = datetime.strptime(date_end, '%Y-%m-%d').date()

filename_prefix = "./Data/apple_stock_"

In [7]:

for single_date in daterange(start_date, end_date):
        
    # set start timestamp
    start_ts = single_date

    # set end timestamp
    end_ts =  single_date + timedelta(days=1)

    # payload rules for v2 api
    rule = gen_request_parameters(query = search_words,
                            results_per_call = 500,
                            start_time = start_ts.isoformat(),
                            end_time = end_ts.isoformat(),
                            tweet_fields = tweet_fields,
                            user_fields = user_fields,
                            expansions='author_id',
                            granularity=None,
                            stringify = False)

    # result stream from twitter v2 api
    rs = ResultStream(request_parameters = rule,
                      max_results=100000,
                      max_pages=1,
                      max_tweets = 10000,
                      **search_creds)

    # number of reconnection tries
    tries = 10

    while True:
        tries -= 1
        try:
                # indicate which day is getting retrieved
            print(f'[INFO] - Retrieving tweets from {str(start_ts)}')

            # get json response to list
            tweets = list(rs.stream())

            # break free from while loop
            break
        except Exception as err:
            if tries == 0:
                raise err
            print(f'[INFO] - Got connection error, waiting 15 seconds and trying again. {tries} tries left.')

            time.sleep(15)

    # parse results to dataframe
    print(f'[INFO] - Saving tweets from {str(start_ts)}')
    file_prefix_w_date = config['filename_prefix'] + start_ts.isoformat()
    outpickle = f'{file_prefix_w_date}.pkl'
    save_object(tweets, outpickle)

[INFO] - Retrieving tweets from 2022-10-02
[INFO] - Saving tweets from 2022-10-02


In [8]:
file = open('./Data/apple_stock_2022-10-02.pkl', 'rb')

# dump information to that file
data = pickle.load(file)

# close the file
file.close()

In [12]:
type(data)

list

In [15]:
data[0]

{'users': [{'username': 'MGuthner',
   'public_metrics': {'followers_count': 3087,
    'following_count': 2251,
    'tweet_count': 1987,
    'listed_count': 75},
   'name': 'Mark W Guthner, CFA',
   'id': '2924274199'},
  {'username': 'CheddarFlow',
   'public_metrics': {'followers_count': 131210,
    'following_count': 82,
    'tweet_count': 29419,
    'listed_count': 2096},
   'name': 'Cheddar Flow',
   'id': '1004137473679679488'},
  {'username': 'BakerofTrades',
   'public_metrics': {'followers_count': 553,
    'following_count': 133,
    'tweet_count': 9682,
    'listed_count': 10},
   'name': 'BakerofTrades',
   'id': '1388556362171535363'},
  {'username': 'astrinvestor',
   'public_metrics': {'followers_count': 506,
    'following_count': 272,
    'tweet_count': 5900,
    'listed_count': 15},
   'name': 'SpaceUranium',
   'id': '1437762101712351236'},
  {'username': 'FibonacciInves1',
   'public_metrics': {'followers_count': 2130,
    'following_count': 33,
    'tweet_count': 30